In [166]:
import pandas as pd
import numpy as np
import json

# 身長,体重,ラベルのCSVデータを読み出す
csv = pd.read_csv("bmi.csv")
# ラベルを三次元のクラスで表す
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))
# データを正規化
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
print(csv)


       height  weight   label  label_pat
0       0.710    0.62     fat  [0, 0, 1]
1       0.710    0.73     fat  [0, 0, 1]
2       0.885    0.61  normal  [0, 1, 0]
3       0.935    0.48    thin  [1, 0, 0]
4       0.765    0.60     fat  [0, 0, 1]
5       0.930    0.59    thin  [1, 0, 0]
6       0.830    0.53  normal  [0, 1, 0]
7       0.965    0.56    thin  [1, 0, 0]
8       0.920    0.43    thin  [1, 0, 0]
9       0.965    0.64    thin  [1, 0, 0]
10      0.775    0.69     fat  [0, 0, 1]
11      0.710    0.63     fat  [0, 0, 1]
12      0.905    0.72  normal  [0, 1, 0]
13      0.945    0.62    thin  [1, 0, 0]
14      0.700    0.62     fat  [0, 0, 1]
15      0.765    0.37    thin  [1, 0, 0]
16      0.655    0.55     fat  [0, 0, 1]
17      0.685    0.68     fat  [0, 0, 1]
18      0.765    0.40    thin  [1, 0, 0]
19      0.680    0.50     fat  [0, 0, 1]
20      0.625    0.73     fat  [0, 0, 1]
21      0.960    0.71  normal  [0, 1, 0]
22      0.735    0.49  normal  [0, 1, 0]
23      0.805   

In [167]:
step=0
rows = csv[step*10:(step+1)*10]
x_pat = rows[["weight","height"]]
y_ans = rows["label_pat"]

csv[0:0]

,height,weight,label,label_pat


In [15]:
import tensorflow as tf


In [168]:
step = 0
rows = csv[1+step*10:(step+1)*10]
x_pat = rows[["weight","height"]]
y_ans = list(rows["label_pat"])

x = tf.placeholder(tf.float32, [None, 2], name="x")
y_ = tf.placeholder(tf.float32, [None, 3], name="y_")

sess = tf.Session()
#sess.run(x, feed_dict={x: x_pat})
sess.run(y_, feed_dict={y_: y_ans})




array([[ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]], dtype=float32)

In [195]:
# データを入れるプレースホルダを宣言
x    = tf.placeholder(tf.float32, [None, 2], name="x") # 身長,体重のデータを入れる
y_ = tf.placeholder(tf.float32, [None, 3]) # 答えのラベルを入れる

# 変数を宣言
W = tf.Variable(tf.random_uniform([2, 3], -1.0, 1.0)); # 重み
b = tf.Variable(tf.zeros([3])); # バイアス
# ソフトマックス回帰を定義
y = tf.nn.softmax(tf.matmul(x, W) + b)

# モデルを訓練する
cross_entropy = -tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y, 1e-10,1.0)))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 正解率を求めるためにテストデータを準備
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])
# 正解率を求める
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(predict, "float"))

# セッションを開始
sess = tf.Session()
sess.run(tf.initialize_all_variables()) #変数を初期化
# テストデータを用いて学習させる
for step in range(5000):
    i = (step * 100) % 1000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 100 == 0:
        e = sess.run(cross_entropy, feed_dict=fd)
        a = sess.run(acc, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "ce=", e, "acc=", a)
        

step= 0 ce= 113.347 acc= 0.3214
step= 100 ce= 83.1781 acc= 0.6926
step= 200 ce= 70.4673 acc= 0.7686
step= 300 ce= 63.2392 acc= 0.8194
step= 400 ce= 58.3766 acc= 0.8506
step= 500 ce= 54.7797 acc= 0.8724
step= 600 ce= 51.9568 acc= 0.8924
step= 700 ce= 49.6508 acc= 0.9016
step= 800 ce= 47.7121 acc= 0.91
step= 900 ce= 46.0467 acc= 0.9186
step= 1000 ce= 44.592 acc= 0.9266
step= 1100 ce= 43.3043 acc= 0.9348
step= 1200 ce= 42.1521 acc= 0.9394
step= 1300 ce= 41.1117 acc= 0.9438
step= 1400 ce= 40.165 acc= 0.9472
step= 1500 ce= 39.2979 acc= 0.9512
step= 1600 ce= 38.4993 acc= 0.9532
step= 1700 ce= 37.7601 acc= 0.9548
step= 1800 ce= 37.0728 acc= 0.957
step= 1900 ce= 36.4313 acc= 0.9588
step= 2000 ce= 35.8303 acc= 0.9608
step= 2100 ce= 35.2657 acc= 0.9624
step= 2200 ce= 34.7336 acc= 0.9632
step= 2300 ce= 34.2309 acc= 0.9644
step= 2400 ce= 33.7549 acc= 0.9646
step= 2500 ce= 33.303 acc= 0.965
step= 2600 ce= 32.8733 acc= 0.9658
step= 2700 ce= 32.4638 acc= 0.9664
step= 2800 ce= 32.0729 acc= 0.9662
step

In [186]:
# データを予測
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(predict, "float"))
r = sess.run(acc, feed_dict={x: test_pat, y_:test_ans})
print(r)

0.811


In [183]:
test = tf.argmax(y, 1)
r = sess.run(test, feed_dict={x: [[70/100, 170/200]]})
print(r)
                                                          

[2]
